In [2]:
import math, datetime, time, random
import numpy as np
import pandas as pd
from datetime import timedelta
import re


def load_data(data_path):
  data = pd.read_csv(data_path)  
  return data

inbound = load_data("inbound_loads.csv")
outbound = load_data("outbound_laods.csv")
weather = load_data("weather.csv")
#For loop to ensure that all pallet data is in the same dataframe
pallet = load_data("Pallet_history_Gold_Spike[0].csv")
for x in range(1, 10):
    pallet = pd.concat([pallet, load_data(f"Pallet_history_Gold_Spike[{x}].csv")])
    print(len(pallet))
trainentest = load_data("demand_kWtrain_val.csv")




1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
4925535


In [3]:
import math, datetime, time, random
# for data visualization
import matplotlib.pyplot as plt
#import missingno #doesn't work, why?!


In [4]:
### function to add time columns to a dataframe

def addtimecol(df, colname): ####input df and colname 
        df[colname] = pd.to_datetime(df[colname])         
        df['year'] = df[colname].dt.year
        df['month'] = df[colname].dt.month
        df['day'] = df[colname].dt.day
        df['hour'] = df[colname].dt.hour
        df['minute'] = df[colname].dt.minute        
        return df

In [5]:
###preprocessing inbound dataframe
#droplevels where carriercode is cancel or None because then you cannot estimate the temp. //its not that many
#droplevels where load_start and truck_signin are both missing because useless info then
inbound = load_data("inbound_loads.csv")
inbound_indexes = inbound[inbound['carrier_code'] == "CANCEL"].index
print(inbound_indexes)
inbound.drop(inbound_indexes, inplace = True)
inbound.dropna(subset = ['customer_code'], inplace = True)
inbound.dropna(axis = 0, how = 'any', thresh = 1, subset = ['load_start_datetime', 'truck_signin_datetime'], inplace = True)


Int64Index([    8,     9,    10,    11,    12,    13,    14,    15,    18,
               23,
            ...
            38864, 38865, 40462, 42264, 43238, 44801, 51019, 51114, 51666,
            51886],
           dtype='int64', length=1996)


In [6]:
df

NameError: name 'df' is not defined

In [7]:
####testing what pallet data says what

###move_category each has their own unique work_types. There is no crossover between them.Inbound moves can be un-load, 
testpal = pd.DataFrame({'from_location_code' : pallet['from_location_code'], 'move_category' : pallet['move_category'], 'received_date' : pallet['received_date'], 'customer_code' : pallet['customer_code'],  'to_location_code' : pallet['to_location_code'],  'work_type' : pallet['work_type'],  'transaction_date' : pallet['transaction_date']    })

outb = testpal[testpal['move_category'] == 'outbound move']
inb = testpal[testpal['move_category'] == 'inbound move']
loc = testpal[testpal['move_category'] == 'location move']
movout = outb['work_type'].unique()
movin = inb['work_type'].value_counts()
movloc = loc['work_type'].unique()
#print('moveout:     ', movout)
#print('movin:     ', movin)
#print('movloc:     ', movloc)

### inbound move can be PUTAWAYS, UN-LOAD and INBOUNDRUN. What does each mean, does one mean that it is coming from the dock? 
inunl = testpal[testpal['work_type'] == 'UN-LOAD'] ##orders mainly coming from going to DR0.. and going to DR0... 
inputa = testpal[testpal['work_type'] == 'PUTAWAYS'] ##orders mainly coming from door numbers, and going to 0... number
ininb = testpal[testpal['work_type'] == 'INBOUNDRUN'] ##orders mainly coming from DR0.., and going to large variety of places, inverter, dr.., 
#print('inunl:     ', inunl['to_location_code'].value_counts())
#print('inputa:     ', inputa['to_location_code'].value_counts())
#print('ininb:     ', ininb['to_location_code'].value_counts())

####what is interesting data for where warm orders go.
        ###move_category: outbound_move = not interesting because orders have already been stored(time between receive date and t is big)
            #inbound_move = ; location_move = 
        ###received_date: skip too old packages because they are in the right temp. to store --> too old = longer than 72h. in?
        ###to_location_code: CPsth = per definitie een outbound move. DRsth = per definitie een outbound move
        ###work_type; MOVE = location_move; PUTAWAYs = inbound, close to entrance; 

In [8]:
#### feature: how many warm/new products are in? at time t
#I didn't include the estimate for temperature

def f_warmproducts(time):                             ##from a single value in trainentest, get the amount of products that 
                                                      ###entered the building within 24 hours
    amount = 0    
    delta = timedelta(days = 1)    
    t = datetime.datetime.strptime(time, '%m/%d/%y %H:%M')
    newtime = delta + t
    inbound['load_start_datetime'] = pd.to_datetime(inbound['load_start_datetime'])  ###make column datetime objects
    
    for index, row in inbound.iterrows():                                      ### amount+1 for each order that entered the building within 24h
        if inbound.at[index, 'carrier_code'] != "CANCEL":
            if t <= inbound.at[index, 'load_start_datetime'] <= newtime :
                amount += measurewarmth(index, row)                            ### use measurewarmth to estimate the warmth that an entry brings in
                
    return amount
                


def measurewarmth(index, row):                                                ###currently, measure warmth uses the weight only to estimate the heat
    weight = inbound.at[index, 'net_weight']
    #medianwarmth(index,row) = 
    return weight


In [ ]:
####using the functions I made

featurestrain = pd.DataFrame({'datetime_local': trainentest['datetime_local'][0:273987]})   ###making a dataframe for features
featurestest = pd.DataFrame({'datetime_local': trainentest['datetime_local'][273988:]})
for index, rows in featurestrain.iterrows(): ###This takes extremely long
   
    featurestrain['f_warmproducts'] = f_warmproducts(featurestrain.at[index, 'datetime_local'])
    
print(features)
    


###using addtimecol to inbound given a certain column
#inbound = addtimecol(inbound, 'load_start_datetime')

In [41]:
####estimating the current warmth of incoming products

###from the inbound datafame, add column 'estimate_temperature' to add the median temp. given the customer code

###make a dictionary with customer code and its median temperature
######get the median middle temperature of each customer code into a dataframe so you can 
warmthdf = pd.DataFrame({'customer_code': inbound['customer_code'], 'carrier_code': inbound['carrier_code'], 'middle_temperature': inbound['middle_temperature']})
    ###filter through missing values/droplevel at missing values
warmthdf = warmthdf.dropna()

warmthdict = {}
for customer in warmthdf.customer_code.unique():      ###per customercode, give the median middle_temperature
    median = warmthdf.loc[warmthdf['customer_code'] == customer]
    warmthdict[f'{customer}']=  median['middle_temperature'].median()

warmthdict['new'] = warmthdf['middle_temperature'].median() ###add median temp for the customer_codes that are new

###iterate over inbound and fill in the estimate_temperature 
for index, row in inbound.iterrows():
    if inbound.at[index, 'middle_temperature'] >-5000: ####fill in the middle_temperature if that is known
        inbound.at[index, 'estimate_temp'] = inbound.at[index, 'middle_temperature']
       
    else: ###else fill in the estimated temp. that belongs to the customer_code, unless it is not in the dict., then fill in 0.0 which is the median temp. of all the orders
        code = inbound.at[index, 'customer_code']        
        median = warmthdict.get(f'{code}', 0.0) 
        inbound.at[index, 'estimate_temp'] = median

warmthdict
###To check the different variances per customer_code: boxplot the different customer_code temperatures
#warmthdf.assign(index=df.groupby('customer_code').cumcount()).pivot('index','customer_code','middle_temperature').plot(kind='box', figsize = [30,30])


{'2537.0': 1.0,
 '8873.0': 0.0,
 '14104.0': -1.5,
 '625.0': 0.0,
 '8187.0': -1.0,
 '5810.0': -1.0,
 '6134.0': -2.0,
 '6910.0': -1.0,
 '899.0': 1.0,
 '7240.0': -1.0,
 '874.0': 0.0,
 '114276.0': 0.0,
 '7475.0': 33.0,
 '1296.0': -3.0,
 '17296.0': -2.0,
 '7289.0': 32.5,
 '2319.0': 3.0,
 '117006.0': 0.0,
 '10377.0': -8.0,
 '1980.0': -1.0,
 '10235.0': -1.5,
 '1059.0': 0.0,
 '3610.0': -1.0,
 '1987.0': -1.0,
 '11383.0': -6.0,
 '615.0': 4.0,
 '8713.0': 0.0,
 '6147.0': -2.0,
 'new': 0.0}

In [133]:
####estimating the current warmth of incoming products using sklearn ###THIS IS NOT WORKING/SHOULD I TRY?
from sklearn.model_selection import train_test_split

###make warmthdf that consists of all the known middle temperature order entries
warmthdf = pd.DataFrame({'customer_code': inbound['customer_code'], 'carrier_code': inbound['carrier_code'], 'middle_temperature': inbound['middle_temperature']})
warmthdf = warmthdf.dropna()

X = warmthdf['customer_code']

y = warmthdf['middle_temperature']
###IDK just split it into test and trainsets
X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)

#print(X_train)
#### its categorical data because X is a category and you then want to estimate the y. 
from sklearn.tree import DecisionTreeRegressor

#regressor = DecisionTreeRegressor(random_state = 0) 
  
# fit the regressor with X and Y data
#regressor.fit(X_train, y_train)


In [77]:
###pallet history contains where packages are going. Which packages are interesting? Testing above. Making it into a feature:

###How many packages are entering a certain area at t as a feature for each location-type, we first make location types to and from
    ###
    ###How much time must be considered? 24-72h for blastfreezer, ideal: between in and out.
    ###What locations to consider? Ignore forks/dock/
    ###what locations can be considered the same, ideal: what locations have same temp.
    
    ##preprocessing the data:
    ###some packages are already in the warehouse for a long time
        #don't consider movements if the time between entranc and t is more than 72 hours because won't cost extra energy 
    



#df = df.assign(movcat = lambda x: (x[0:3]))
#df = df.assign(movcat = lambda x: (x['to_location_code'][0:3])

df = pallet[10000:50000]
for index, row in df.iterrows():  
        a = df.at[index, 'to_location_code']
        b = df.at[index, 'from_location_code']
        df.loc[index, 'frommovcat'] = b[0:3]
        df.loc[index, 'tomovcat'] = a[0:3]
df['tomovcat'].value_counts()


###per movement type I will make one feature that defines how many pallets are moved towards a location. 
    #df = df.assign(Product=lambda x: (x['Field_1'] * x['Field_2'] * x['Field_3']))
    

###divide moving history into different category of areas. 
#mapping = {'(/^FORK/gm)': 'Fork', '(/^CFORK/gm)': 'CFORK', '(/^30/gm)' : '30', '(/^CP/gm)' : 'CP',  '(/^50/gm)' : '50',  '(/^15/gm)' : '15', '(/^11/gm)' : '11',  '(/^01/gm)' : '01' 
           

    ### amount+1 for each order that entered the building within 24h
#            if t <= inbound.at[index, 'load_start_datetime'] <= newtime :
#                amount += measurewarmth(index, row)                            ### use measurewarmth to estimate the warmth that an entry brings in
                
#    return amount


003    8608
FOR    4982
004    4347
015    2640
005    2607
CFO    2459
002    2216
DR0    2051
012    1911
009    1370
010    1199
CP9    1088
CP2    1037
011     913
INV     712
001     599
CP1     399
CP6     241
WRA     221
014     151
013      58
DR9      49
CP7      45
SPJ      41
263      26
000      11
CP4       8
DR5       7
UNA       4
Name: tomovcat, dtype: int64

In [ ]:

###pallet types of locations:
#003    2130
#FOR    1177 
#012    1116
#011    1116
#004    1024
#005     619
#CP9     566
#DR0     483
#CP2     371
#CFO     282
#010     200
#009     174
#INV     142
#SPJ     133
#CP1     126
#CP6     109
#002      69
#WRA      60
#001      47
#015      40
#CP7       8
#CP4       5
#DR9       2
